In [2]:
# !pip install --upgrade pip
# !pip3 install pandas
# !pip3 install scikit-learn
# !pip3 install optuna
# !pip install --upgrade wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.2 MB/s  0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 13.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 13.6 MB/s  0:00:02m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.6 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 12.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [optuna]2m6/7 [optuna]]my]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 13.6 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.

In [3]:
# Warnings configuration
import warnings
warnings.filterwarnings('ignore')

# General Libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import time

# Neural Network Components
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from keras import backend as K

#Data preprocessing
from sklearn.model_selection import train_test_split
from PIL import Image

# Experiments Managers
import optuna
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

In [4]:
# wandb.login()

In [5]:
#Import the modules
from complete_model_creator import NeuralNetworkConstructor
from complete_model_creator import ModelBuilder
from complete_model_creator import  ImageDataGenerator

In [7]:
# Define directory path and parameters
data_dir = '/tf/Tarea_6/plant-pathology-2020-fgvc7/images' # e.g., 'data/train'
image_size = (256//2, 384//2)
batch_size = 10

# Create a dataset for training
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir+'/train',
    labels='inferred',  # Inferred from subdirectory names
    label_mode='categorical', # or 'binary' or 'int'
    image_size=image_size,
    interpolation='bilinear',
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.7,
    subset='training',
    seed=42 # for reproducibility
)

# Optional: Create a validation set by specifying validation_split and subset
test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir+'/test',
    labels='inferred',
    label_mode='categorical',
    image_size=image_size,
    interpolation='bilinear',
    batch_size=batch_size,
    shuffle=True,
    validation_split=0.7,
    subset='training',
    seed=42,
)

# Preprocessing 
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Prepare for performance
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 1092 files belonging to 4 classes.
Using 328 files for training.
Found 546 files belonging to 4 classes.
Using 164 files for training.


In [6]:
number_of_classes = 4

In [7]:
def objective(trial):
    # Early stopping callback
    early_stopping = keras.callbacks.EarlyStopping(monitor ='val_loss',
                                                   min_delta = 0.001,
                                                   patience = 50, 
                                                   restore_best_weights = True,
                                                   mode = "auto",
                                                   verbose = 1,
                                                   baseline = None)

    builder = ModelBuilder(trial, 
                           inputs=keras.Input((image_size[0],
                                               image_size[1],
                                               3)),
                           total_classes=number_of_classes)
    
    model = builder.get_model()
    params = builder.get_params(1)
    
    run = wandb.init(
        project="Plant_Pathology_Test",
        group="mini_train",
        name=f"Trial_{trial.number}",
        config=params
    )
    
    # Train the model with data generators
    history = model.fit(
        train_ds,
        epochs=10, 
        batch_size=batch_size,  # Use the tuned batch_size
        verbose=0,
        validation_data=test_ds,
        callbacks=[
            WandbMetricsLogger(log_freq=1),
            early_stopping
        ]
    )
    
    # Finish wandb run
    run.finish()
    # Evaluate on validation set (not test set for hyperparameter optimization)
    val_loss, val_accuracy = model.evaluate(test_ds, verbose=0)
    return val_loss  # Optimize on validation loss

In [8]:
study = optuna.create_study(direction = "minimize")
study.optimize(objective, n_trials = 30)

[I 2025-11-03 19:19:02,306] A new study created in memory with name: no-name-94816408-1655-4edb-8a6b-9e5907775121
wandb: Currently logged in as: luisedu04 (luisedu04-buap) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/usr/local/lib/python3.11/dist-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


[W 2025-11-03 19:21:04,437] Trial 0 failed with parameters: {'conv_blocks': 3, 'maxpooling_rate': 2, 'bottleneck_length': 2, 'filters': 128, 'bottleneck_dropout': 0.37941509752545677, 'optimizer': 'Nadam', 'Dense_units': 256, 'Dense_activation': 'tanh', 'Dense_dropout': 0.41839740267655257, 'BN_activation0': 'tanh', 'BN_regularizer0': 'L1L2', 'BN_activation1': 'relu', 'BN_regularizer1': 'L1L2', 'DW_activation0': 'relu', 'DW_regularizer0': 'L1L2', 'UW_activation0': 'tanh', 'UW_regularizer0': 'L2', 'DW_activation1': 'tanh', 'DW_regularizer1': 'L2', 'UW_activation1': 'relu', 'UW_regularizer1': None, 'DW_activation2': 'tanh', 'DW_regularizer2': 'L1L2', 'UW_activation2': 'tanh', 'UW_regularizer2': None, 'DW_activation3': 'relu', 'DW_regularizer3': 'L2', 'UW_activation3': 'relu', 'UW_regularizer3': 'L1', 'DW_activation4': 'tanh', 'DW_regularizer4': 'L1L2', 'UW_activation4': 'relu', 'UW_regularizer4': None, 'DW_activation5': 'relu', 'DW_regularizer5': 'L2', 'UW_activation5': 'relu', 'UW_regul

KeyboardInterrupt: 